In [6]:
"""
This script applies the event-based master recession curve watertable fluctuation method (e.g., Nimmo and Perkins 2018) 
to estimate recharge from synthetic hydrographs. The method is implemented on a series of generated 
recharge events, and recharge estimates are compared to known values. The output is the relative error (%) of the 
recharge estimate, calculated as described in Becke et al. (2025).
"""
import os
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit

# Configuration
FOLDER_NAME = "EMR_Output_Check"
BASE_DIR = os.path.join(os.path.expanduser("~"), "workspace", "WTF", "OutputFiles", FOLDER_NAME)
try:
    # Works if script is run as a .py file
    SCRIPT_DIR = os.path.dirname(__file__)
except NameError:
    # Fallback for Jupyter notebooks where __file__ is not defined
    SCRIPT_DIR = os.getcwd()

INPUT_DIR = os.path.abspath(os.path.join(SCRIPT_DIR, '..', 'data'))


os.makedirs(BASE_DIR, exist_ok=True)
os.chdir(BASE_DIR)

# Constants
sci = [0.1, 0.25, 0.5, 0.75, 0.9]
time = np.linspace(0, 400., 401)
EVENT_TIMES = {
    1: (30, 90, 120),  #t1, t2, t3
    2: (120, 210, 240), #t3, t4, t5
    3: (240, 360, 390)  #t5, t6, t7
}

def extract_recession_rate(data):
    return [i for i in data if i < 0]

def linear_fit(rec_values, A, B):
    return -A * rec_values + B

def errors_emr(itr, time, events):
    errors_all = {event_id: [] for event_id in events.keys()}

    for j in range(1, itr + 1):
        if j in {281, 949}:
            continue

        try:
            wl_file = os.path.join(INPUT_DIR, f"hydro_{j:04d}.csv")
            var_file = os.path.join(INPUT_DIR, f"var_{j:04d}.csv")

            df_wl = pd.read_csv(wl_file, header=None).to_numpy().flatten()
            df_var = pd.read_csv(var_file, header=None).to_numpy()

            dh_dt = np.gradient(df_wl)
            rec_rate = np.array(extract_recession_rate(dh_dt))
            loc_rec = np.where(dh_dt < 0)
            rec_wl = df_wl[loc_rec]

            params, _ = curve_fit(linear_fit, rec_wl, rec_rate, maxfev=8000, method='trf')
            K1, K2 = params

            for event_id, (t_start, t_mid, t_end) in events.items():
                if t_end >= len(df_wl):
                    continue  # Skip if t_end is out of bounds

                # Use water level at the end of recession as the starting point
                wl_proj = df_wl[t_mid]

                # Dynamically compute number of steps to project
                projection_steps = t_end - t_mid
                if projection_steps <= 0:
                    print(f"Invalid projection length for event {event_id} in hydrograph {j}")
                    continue

                for _ in range(projection_steps):
                    wl_proj += K2 + wl_proj * (-K1)

                # Use actual WL at the end of projection period to compute recharge
                delta_h = df_wl[t_end] - wl_proj
                delta_t = time[t_end] - time[t_mid]

                recharge_est = (delta_h / delta_t) * df_var[3][0]
                true_recharge = df_var[0][0]
                error = ((recharge_est - true_recharge) / true_recharge) * 100

                errors_all[event_id].append(error)


        except Exception as e:
            print(f"Error processing hydrograph {j}: {e}")
            continue

    return errors_all

if __name__ == "__main__":
    errors = errors_emr(1000, time, EVENT_TIMES)
    for event_id, error_list in errors.items():
        output_file = f'EMR_Error_Event{event_id}.csv'
        np.savetxt(output_file, error_list, delimiter=',', header=f'EMR_Error_Event{event_id}', comments='')
        print(f"Output saved to {output_file}")

Output saved to EMR_Error_Event1.csv
Output saved to EMR_Error_Event2.csv
Output saved to EMR_Error_Event3.csv
